# Swin Transformer: Hierarchical Vision Transformer using Shifted Windows  
- 논문 리뷰

- 이 논문은 Vision Transformer가 제한적인 application을 가진 것에 대해 단점으로 삼으며 더 유연하고 효율적이며 다양한 task에 backbone으로 쓸 수 있는 Swin Transformer를 제안한다  

## Abstract  
- 언어에서 비전으로 Transformer를 적용할 때 어려운 점이 있다  
- text와 image의 domain의 scale이 너무 다르다는 것이다  
- image는 저해상도부터 고해상도가 있고 이미지 내에 크고 작은 object가 다양하게 존재한다  
- 그래서 이를 해결하기 위해 shifted window를 이용하는 hierarchical Transformer를 제안한다  
- 본 논문에서 제안하는 shifted window 방식은 non-overlapping으로 계산 복잡성을 제한해 효율성을 가져온다    
- hierarchical architecture는 다양한 모델에서 유연하게 동작되며 image의 size에 따라 선형적인 연산 복잡성을 가지고 있다  
- 각 task에서 Swin Transformer를 backbone으로 사용한 결과 classification, detection, segmentation 등에서 SOTA 모델을 능가하는 성능을 보여준다  
- 또한 backbone으로 Transformer 기반 모델의 잠재력을 보여준다  
- hierarchical design과 shifted window 방식은 all-MLP architecture에서 유익하다는 것을 보여준다   

## Introduction   
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/figure1.JPG?raw=true" width=800>  
  
- 본 논문에서 우리는 vision의 CNN처럼 general-purpose backbone의 역할을 할 수 있도록 Transformer의 applicability를 확장하는 것을 추구한다  
- 우리는 language에서 visual domain으로 high performance를 transfer하는 데 중요한 차이점 두 가지를 관찰했다  
- 하나는 Transformer 기반 모델에서 token은 고정된 scale을 가지는데 이는 vision application에 적합하지 않다  
- 또 다른 차이점은 text의 구절에서 words에 비해 이미지는 훨씬 더 많은 resolution을 가진다는 점이다  
- vision high level task는 pixel level로 dense prediction을 필요로 한다  
- 그러나 Transformer는 image size에 따라 selt-attention의 연산 복잡도가 quadratic하게 늘어난다  
- 이러한 issue를 극복하기 위해 우리는 general-purpose Transformer backbone인 Swin Transformer를 제안한다  
- hierarchical feature map으로 구성되어 있으며 image size에 따라 연산량이 linear하게 증가한다  
- non-overlapping window로 selt-attention이 수행되고 각 window에서 patch의 수는 고정되어 있어 linear 연산량을 가진다  
  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/figure2.JPG?raw=true" width=800>  

- Swin Transformer의 주요 디자인은 연속적인 selt-attention layer 사이에 window partition을 이동시키는 것이다  
- shifted window는 이전 레이어의 window를 연결하며 이들 사이의 connection을 제공해 성능을 향상시킨다  
- 이러한 방법은 효율적이며 window 내의 모든 query patch들은 같은 key set을 공유해 memory access를 용이하게 한다  
- 우리는 실험을 통해 sliding window method보다 shifted window method가 더 낮은 대기 시간을 가진다는 것을 보여준다  
- 또한 shifted window가 all-MLP architecture에 대해 유익하다는 것을 보여준다  
- Swin Transformer는 ViT, DeiT 그리고 ResNeXt를 backbone으로 사용한 다양한 task의 이전 SOTA 모델들보다 더 나은 성능을 보여주고 있다  

## Related Work  
**Self-attention based backbone architectures**  
- 기존 Transformer의 self-attention은 sliding window 내의 각 픽셀들을 빠르게 최적화했다  
- 그러나 이 방식은 CNN보다 더 많은 memory access를 소비하며 더 느렸다  
- 그래서 sliding window 대신 shifted window를 사용해 일반적인 하드웨어에도 적용하며 효율적인 구현을 하려 한다  

**Transformer based vision backbones**  
- ViT는 좋은 성능을 얻기 위해선 JFT-300M과 같은 large scale dataset으로 사전 훈련을 진행해야 한다  
- 반면 DeiT는 ImageNet 1K만으로도 충분해 ViT보다 더 효율적이다  
- 그러나 이러한 모델들의 architecture는 dense vision task에 general-purpose network로는 적합하지 않다  
- 또한 이미지의 resolution이 커질수록 연산량이 quadratic하게 늘어난다  
- 실험을 통해 우리는 Swin Transformer의 architecture가 다양한 task에서 speed와 accuracy 사이의 trade-off를 적절히 이뤄냈음을 보여준다  
- 우리와 비슷하게 multi scale의 feature map을 이용한 Transformer 모델도 있었다  
- 이 연구 역시 이미지 사이즈에 따라 연산량이 quadratic하게 증가하는 반면 우리 모델은 linear하게 증가하므로 효율적인 구조를 가지고 있다  

## Method  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/figure3.JPG?raw=true" width=1000>

### Overall Architecture  
- figure 3에서 Swin Transformer는 RGB image를 input으로 받으면 ViT처럼 non-overlapping으로 patch를 잘라낸다  
- 각 patch들은 token처럼 다뤄지며 이들은 concat된다  
- patch size는 $4\times 4$로 설정했으며 각 patch의 feature dimension은 $4\times 4\times 3=48$이다  
- 그리고나서 linear embedding layer가 적용되며 임의의 dimension C로 projection된다  
- 각 Swin Transformer Block은 이러한 patch token에 적용된다  
- 또한 이 과정에서 token의 수는 유지되며 이를 stage 1이라 한다  
- hierarchical representation을 생성하기 위해 네트워크가 깊을수록 patch merging layer에 의해 token의 수가 줄어든다  
- 첫 번째 patch merging layer는 $2\times 2$ 인접 patch끼리 합치면 채널이 4배가 되어 4C dimension이 되며 lineawr layer를 적용시킨다  
- 그리고 나면 token의 수는 4배로 줄어들고 resolution은 2배 줄어들며 output channel의 수는 2C dimension으로 설정한다  
- 그 후에 Swin Transformer Block이 적용되며 resolution은 $\frac{H}{8}\times\frac{W}{8}$로 유지된다  
- 이를 stage 2라 하며 이러한 과정을 2번 더 반복해 stage 3와 stage 4를 얻는다  
- 각각 output resolution은 $\frac{H}{16}\times\frac{W}{16}$, $\frac{H}{32}\times\frac{W}{32}$에 해당한다  
- 이러한 stage들로 VGG나 ResNet처럼 같은 resolution의 hierarchical feature를 만들 수 있다  
- 결론적으로 우리가 제안한 architecture는 다양한 task에서 backbone으로 사용할 수 있다  
  
**Swin Transformer block**  
- Swin Transformer Block은 shifted window 기반 MSA module과 그 사이에 GELU와 2개의 MLP로 구성된다   
- Layer Norm은 각각 MSA module과 MLP를 거치기 전에 적용되며 그 이후에 residual connection이 적용된다  
   
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/figure4.JPG?raw=true" width=900>

### Shifted Window based Self-Attention  
- Standard Transformer는 self-attention을 처리하는 과정에서 token과 다른 모든 token 사이의 관계를 계산한다  
- 이는 vision task에서 이미지 해상도가 높을수록 연산량이 quadratic해진다  

**Self-attention in non-overlapped windows**  
- 각 window가 $M\times M$ patch들을 포함한다고 가정하면 global MSA module과 $h\times w$ patch의 이미지를 기반으로 하는 window의 연산량은 다음과 같다  
$$\Omega\left(MSA\right)=4hwC^2+2\left(hw\right)^2C$$    
$$\Omega\left(W-MSA\right)=4hwC^2+2M^2hwC$$  
- MSA의 연산량은 이미지 사이즈 h, w에 quadratic하게 증가하는 반면 W-MSA는 그보다 더 작은 patch size인 M에 영향을 받는다  

**Shifted window partitioning in successive blocks**  
- window 기반 self-attention module은 window 사이의 connection이 부족하다  
- non-overlapping window의 효율적인 연산을 유지하면서 cross window connection 방식인 shifted window partitioning 접근법을 소개한다  
- figure 2에서 첫 번째 module은 일반적인 window partitioning 방식을 사용한다  
- $8\times 8$ feature map에서 $4\times 4$ size의 window 4개로 균등하게 분할한다  
- 그러면 다음 module은 분할된 window에서 $\left(\lfloor\frac{M}{2}\rfloor, \lfloor\frac{M}{2}\rfloor\right)$ pixel 만큼 대체해 이전 layer에서 window 배열을 옮긴다    
- figure 3 (b)를 수식으로 표현하면 다음과 같다  
$$\hat{\textbf{z}}^{l}=W-MSA\left(LN\left(\textbf{z}^{l-1}\right)\right)+\textbf{z}^{l-1}$$  
$$\textbf{z}^l=MLP\left(\left(\hat{\textbf{z}}^l\right)\right)+\hat{\textbf{z}}^l$$  
$$\hat{\textbf{z}}^{l+1}=SW-MSA\left(LN\left(\textbf{z}^l\right)\right)+\textbf{z}^l$$    
$$\textbf{z}^{l+1}=MLP\left(LN\left(\hat{\textbf{z}}^{l+1}\right)\right)+\hat{\textbf{z}}^{l+1}$$  
- 이때 $\hat{\textbf{z}}^l$과 $\textbf{z}^l$은 각각 MSA와 MLP module의 output에 해당한다  
- 이처럼 shifted window partitioning 방식은 각 task에서 효율적으로 작동하며 좋은 성능을 보여준다  

**Efficient batch computation for shifted configuration**  
- shifted window partitioning issue는 window가 더 많아진다는 것이다  
- 그리고 몇몇 window는 $M\times M$보다 작아진다  
- 이를 해결할 수 있는 naive한 solution은 attention 연산 과정에서 $M\times M$보다 작은 window에 대해서 padding을 하는 것이다  
- 그래서 우리는 figure 4와 같이 cyclic-shifting을 수행하는 더 효율적인 batch computation 접근법을 제안한다  
- 이렇게 shift를 하고 나면 batched window는 feature map에서 인접하지 않은 각 sub-window로 이루어져 있다  
- 그래서 masking mechanism은 각 sub-window 내에서 self-attention 연산을 제한하기 위해 사용된다  
- cyclic-shift를 이용하면 batched window의 수는 regular window partition 방식과 동일하게 유지되므로 효과적이다  

**Relative position bias**  
- 우리는 각 head에 relative position bias를 다음과 같이 연산에 포함시킨다  
$$Attention\left(Q, K, V\right)=SoftMax\left(QK^T/\sqrt{d}+B\right)V$$  

- 우리는 bias 없이 혹은 absolute position embedding을 사용한 것에 비해 상당한 개선을 이뤘다  
- absolute position embedding을 input에 추가한 결과, 성능이 약간 떨어져 우리는 이를 채택하지 않았다  

### Architecture Variants  
- The window size is set to $M=7$ by default  
- The query dimension of each head is $d=32$  
- The expansion layer of each ML is $\alpha=4$  
    - Swin-T : $C=96$, layer number = $\left\{2,2,6,2\right\}$  
    - Swin-S : $C=96$, layer number = $\left\{2,2,18,2\right\}$  
    - Swin-B : $C=128$, layer number = $\left\{2,2,18,2\right\}$  
    - Swin-L : $C=192$, layer number = $\left\{2,2,18,2\right\}$  
    - 이때 C는 first stage에서 hidden layer의 channel 수를 의미한다  

## Experiments  
- 우리는 실험을 통해 Swin Transformer를 classification, object detection, semantic segmentation 3개의 task에서 SOTA 모델들과 비교를 했다  

### Image Classification on ImageNet-1K  
**Settings**  
- 1.28M training images, 50K validatoin images and 1000 classes  
- 우리는 2개의 training setting을 고려했다  
    - Regular ImageNet-1K training  
        - AdamW optimizer  
        - 300 epochs  
        - cosine decay learning rate scheduler and 20 epochs of lineawr warm-up  
        - batch size of 1024  
        - initial learning rate of 0.001  
        - weight decay of 0.05  
        
    - Pre-training on ImageNet-22K and fine-tuning on ImageNet-1K  
        - pre-train on the large ImageNet-22K dataset  
        - 14.2M images and 22K classes  
        - AdamW optimizer  
        - 90 epochs  
        - learning decay learning rate scheduler with a 5-epoch linear warm-up  
        - batch size of 4096  
        - initial learning rate of 0.001  
        - In ImageNet-1K fine-tuning, train the models for 30 epochs with a batch size of 1024, learning rate of 1e-5, and weight decay of 1e-8  
        
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/table1.JPG?raw=true" width=600>
        
**Results with regular ImageNet-1K training**  
- Transformer 기반 SOTA 모델인 DeiT와 비교를 했다  
- DeiT-S에 비해 Swin-T가 정확도가 1.5% 더 높다  
- CNN SOTA 모델인 RegNet과 EfficientNet과 비교해본 결과  
- Swin Transformer가 speed-accuracy trade-off를 가장 잘 극복했다  

**Results with ImageNet-22K pre-training**  
- Swin-B의 경우, 1.8~1.9% 더 높은 정확도를 얻었다  
- 이전 SOTA 모델과 비교해본 결과, Swin-B가 86.4%로 2.4% 더 높은 성능을 가진다  
- 또한 speed-accuracy의 trade-off도 가장 잘 극복했다  

### Object Detection on COCO  
**Settings**  
- COCO 2017 dataset with 118K training, 5K validation and 20K test images  
- AdamW optimizer  
- initial learning rate of 0.0001  
- weight decay of 0.05  
- batch size of 16  
- 3x schedule (36 epochs)  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/table2.JPG?raw=true" width=600>

**Comparision to ResNe(X)t**  
- Swin-T backbone이 ResNet-50 backbone을 사용한 모델보다 3.4~4.2 더 높은 box AP를 얻었다  
- 또한 ResNeXt101-64x4d 보다 3.6 box AP와 3.3 mask AP 더 높은 성능을 가진다  

**Comparision to previous state-of-the-art**  
- 우리의 best model은 58.7 box AP와 51.1 mask AP를 가진다  
- 이전 SOTA 모델보다 각각 2.7 box AP, 2.6 mask AP 더 좋은 성능을 보인다  

### Semantic Segmentation on ADE20K  
**Settings**  
- 20K train images, 2K validation images and 3K test images  
- AdamW optimizer  
- initial learning rate of 6e-5  
- weight decay of 0.01  
- a scheduler that uses linear learning rate decay and a linear warmup of 1500 iterations  
- trained on 8 GPUs with 2 images per GPU for 160K iterations  
- Augmentation:  
    - random horizontal flipping  
    - random re-scaling with ratio range [0.5, 2.0]  
    - random photometric distortion  
    - Stochastic depth with ratio of 0.2  
    - input image size is 512 x 512 
    - Swin-B and Swin-L are pre-trained on ImageNet-22K  
    - and trained with the input size of 640 x 640  
    
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/table3.JPG?raw=true" width=600>
    
**Results**  
- Swin-S는 DeiT-S 보다 5.3 mIoU 더 높은 성능을 기록했다  
- 또한 ResNet-101보다 4.4 mIoU 더 높으며 ResNeSt-101보다 2.4 mIoU 더 높은 성능을 기록했다  
- ImageNet-22K에서 pre-trained된 Swin-L의 경우, val set에서 53.5 mIoU를 기록하고 이전 SOTA 모델보다 3.2 mIoU 더 높은 성능을 보여준다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/table4.JPG?raw=true" width=500>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/SwinTransformer/figure/table5.JPG?raw=true">

### Ablation Study  
- 이 section에서는 위의 각 task에 따라 fine-tuning된 Swin Transformer의 성능을 분석한다  

**Shifted windows**  
- table 4에 shifted window의 효과를 보여주고 있다  
- shifted window partitioning는 single window partitioning보다 좋은 성능을 보여준다  
- 각 task에서 1.1%, 2.8 box AP, 2.2 mask AP, 2.8 mIoU 더 좋은 성능을 보여준다  

**Relative position bias**  
- table 4에서 각각 다른 position embedding approach들의 성능을 비교하고 있다  
- relative position bias를 사용한 Swin-T는 각각 1.2%, 1.3~1.5 box AP, 1.1~1.3 mask AP, 2.3~2.9 mIoU 더 좋은 성능을 보여주고 있다  
- absolute position embedding은 classification에서는 0.4%의 정확도가 올랐지만 다른 task에선 약간 떨어졌다  
- 우리는 general-purpose modeling을 위해선 여전히 translation invariance가 필요하며 dense prediction을 위해선 inductive bias가 선호된다는 것을 발견했다  

**Different self-attention methods**  
- table 5에서 각각 다른 self-attention computation method들을 비교하고 있다  
- 우리의 cyclic implementation은 naive padding보다 하드웨어를 더 효율적으로 사용한다  
- 전체적으로 cyclic method가 padding method 보다 13~18% 정도로 더 효율적인 FPS를 보여주고 있다  
- sliding window보다 shifted window를 사용한 것이 속도 측면에서도 더 빠르다  
- 정확도 역시 조금 더 좋은 성능을 보여주고 있다  

## Conclusion  
- 이 논문은 hierarchical feature representation을 사용하는 새로운 vision transformer인 Swin Transformer를 제안한다  
- Swin Transformer는 기존 모델에 비해 computational complexity가 이미지 사이즈에 따라 linear하게 증가하는 효율적인 측면이 있다  
- 이에 따라 COCO object detection과 ADE20K semantic segmentation에서 SOTA performance를 보여주고 있다  
- 우리는 Swin Transformer가 vision과 language task 구분 없이 통합된 model이 되기를 원한다  
- Swin Transformer의 key point는 self-attention 기반 shifted window이며 이는 vision problem에서 효율적인 성능을 보여줬다  